In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp


In [5]:
# Get the exceess returns data
returndata = pd.read_csv('data/return_pf_excess_ret_beta.csv', index_col=0)

returndata.head()

,date,permno,ret,mktcap,rf,excess_ret,benchmark_excess_ret,beta_5,beta_10,beta_20,beta_30
0,192601,10006,0.032732,66150.00,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,192601,10022,0.017857,11400.00,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,192601,10030,0.161667,27183.00,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,192601,10049,0.141892,20875.00,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,192601,10057,-0.035714,5906.25,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [4]:
n = 500
beta = returndata[["beta_5"]]
V = np.random.normal(size=(n,n))
beta

,beta_5
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
4652772,1.289819
4652773,1.119160
4652774,0.640248
4652775,2.209166


In [ ]:
def FactorMimickingPortOpt_MV(alpha, beta, gamma, V):
    '''
    output the optimal weights for the factor mimicking portfolio
    - only constraint is beta neutral 
    '''
    n = V.shape[0]
    x = cp.Variable(n)
    
    obj = cp.Maximize(x.T@alpha - gamma/2*cp.quad_form(x,V))
    constraints = [x.T@beta == 0]
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.GUROBI)
    return x.value, prob.value

def FactorMimickingPortOpt_SR(alpha, beta, gamma, V):
    '''
    output the optimal weights for the factor mimicking portfolio
    - only constraint is beta neutral 
    '''
    n = V.shape[0]
    z = cp.Variable(n)
    kappa = cp.Variable(1)

    obj = cp.Minimize(cp.quad_form(z,V))
    constraints = [z.T@beta == 0, z.T@alpha == 1, np.ones(n)@z-kappa==0, kappa>=0, ]
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.GUROBI)
    return z.value/ kappa.value  
